In [1]:
import os
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pickle, datetime

SCOPES = ['https://www.googleapis.com/auth/calendar']
creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

service = build('calendar', 'v3', credentials=creds)

In [2]:
from pymongo import MongoClient
from utils_mongo import MongoUtils as mu
from utils import CalendarTools

import re
from datetime import datetime
import pytz

In [3]:
collection = mu.connect_to_mongodb("agent_calendar", "exec_ai")

### Create

In [5]:
result2 = """{
"event_summary": "Team meeting",
"event_location": "N/A",
"start_time": "2024-03-02T20:00:00+05:30",
"end_time": "2024-03-02T22:00:00+05:30",
"email_reminder_minutes": 1440,
"popup_reminder_minutes": 180
}"""

result_dict = eval(result2)
print(result_dict)

{'event_summary': 'Team meeting', 'event_location': 'N/A', 'start_time': '2024-03-02T20:00:00+05:30', 'end_time': '2024-03-02T22:00:00+05:30', 'email_reminder_minutes': 1440, 'popup_reminder_minutes': 180}


In [6]:
created_event = CalendarTools.create_event(service, result_dict.get('event_summary'), result_dict.get('event_location'), result_dict.get('start_time'), result_dict.get('end_time'), int(result_dict.get('email_reminder_minutes')), int(result_dict.get('popup_reminder_minutes')))
eventId = re.sub(r'@.*', '', created_event)
print(eventId)

event = CalendarTools.list_events(service, event_id=eventId)

Event created: https://www.google.com/calendar/event?eid=NW1yMWlrZTBob3EyNDd1cTU1dnJhZzRjOXMgZ3VudWJhbnNhbDEyOUBt
5mr1ike0hoq247uq55vrag4c9s


In [8]:
print(event[0])

{'kind': 'calendar#event', 'etag': '"3414851391240000"', 'id': '5mr1ike0hoq247uq55vrag4c9s', 'status': 'confirmed', 'htmlLink': 'https://www.google.com/calendar/event?eid=NW1yMWlrZTBob3EyNDd1cTU1dnJhZzRjOXMgZ3VudWJhbnNhbDEyOUBt', 'created': '2024-02-08T20:54:55.000Z', 'updated': '2024-02-08T20:54:55.620Z', 'summary': 'Team meeting', 'description': "A chance to hear more about Google's developer products.", 'location': 'N/A', 'creator': {'email': 'gunubansal129@gmail.com', 'self': True}, 'organizer': {'email': 'gunubansal129@gmail.com', 'self': True}, 'start': {'dateTime': '2024-03-02T20:00:00+05:30', 'timeZone': 'Asia/Kolkata'}, 'end': {'dateTime': '2024-03-02T22:00:00+05:30', 'timeZone': 'Asia/Kolkata'}, 'iCalUID': '5mr1ike0hoq247uq55vrag4c9s@google.com', 'sequence': 0, 'reminders': {'useDefault': False, 'overrides': [{'method': 'email', 'minutes': 1440}, {'method': 'popup', 'minutes': 180}]}, 'eventType': 'default'}


In [9]:
mu.save_new_db(event = event[0], collection=collection)

Added event to DB


----

### Modify

In [17]:
# Define function to prompt user for confirmation of events
def confirm_event(events):
    print("Found the following events:")
    for i, event in enumerate(events):
        print(f"{i + 1}. {event['summary']} - {event['start']['dateTime']} to {event['end']['dateTime']}")

    while True:
        choice = input("Enter the number of the correct event (or '0' to cancel): ")
        if choice == '0':
            return None
        try:
            choice = int(choice)
            if 1 <= choice <= len(events):
                return events[choice - 1]
            else:
                print("Invalid choice. Please enter a valid number.")
        except ValueError:
            print("Invalid input. Please enter a number.")

In [18]:
# Example usage
summary = ""
start_date = "2024-03-02" #YYYY-MM-DD format
end_date = "" # YYYY-MM-DD format
location = ""

events = mu.list_events_db(collection, summary, start_date, end_date, location)

if events:
    selected_event = confirm_event(events)
    if selected_event:
        print("Confirmed event:")
        print(selected_event)
else:
    print("No events found matching the criteria.")

Found the following events:
1. Team meeting - 2024-03-02T20:00:00+05:30 to 2024-03-02T22:00:00+05:30
Confirmed event:
{'_id': ObjectId('65c53fbc22cc11c0ce8c5b26'), 'kind': 'calendar#event', 'etag': '"3414851391240000"', 'id': '5mr1ike0hoq247uq55vrag4c9s', 'status': 'confirmed', 'htmlLink': 'https://www.google.com/calendar/event?eid=NW1yMWlrZTBob3EyNDd1cTU1dnJhZzRjOXMgZ3VudWJhbnNhbDEyOUBt', 'created': '2024-02-08T20:54:55.000Z', 'updated': '2024-02-08T20:54:55.620Z', 'summary': 'Team meeting', 'description': "A chance to hear more about Google's developer products.", 'location': 'N/A', 'creator': {'email': 'gunubansal129@gmail.com', 'self': True}, 'organizer': {'email': 'gunubansal129@gmail.com', 'self': True}, 'start': {'dateTime': '2024-03-02T20:00:00+05:30', 'timeZone': 'Asia/Kolkata'}, 'end': {'dateTime': '2024-03-02T22:00:00+05:30', 'timeZone': 'Asia/Kolkata'}, 'iCalUID': '5mr1ike0hoq247uq55vrag4c9s@google.com', 'sequence': 0, 'reminders': {'useDefault': False, 'overrides': [{'meth

In [15]:
####### ask the user in a natural language way

In [4]:
event_id = selected_event.get('id')

In [12]:
updated_event = CalendarTools.modify_event(service, event_id, event_summary='', event_location='',
                           start_time='2024-03-02T20:00:00+05:30',end_time='',
                           state=3
                        )

Event updated: https://www.google.com/calendar/event?eid=NW1yMWlrZTBob3EyNDd1cTU1dnJhZzRjOXMgZ3VudWJhbnNhbDEyOUBt


In [13]:
updated_event[0]

{'kind': 'calendar#event',
 'etag': '"3414855094589000"',
 'id': '5mr1ike0hoq247uq55vrag4c9s',
 'status': 'confirmed',
 'htmlLink': 'https://www.google.com/calendar/event?eid=NW1yMWlrZTBob3EyNDd1cTU1dnJhZzRjOXMgZ3VudWJhbnNhbDEyOUBt',
 'created': '2024-02-08T20:54:55.000Z',
 'updated': '2024-02-08T21:56:38.969Z',
 'summary': 'Team meeting',
 'description': "A chance to hear more about Google's developer products.",
 'location': 'N/A',
 'creator': {'email': 'gunubansal129@gmail.com', 'self': True},
 'organizer': {'email': 'gunubansal129@gmail.com', 'self': True},
 'start': {'dateTime': '2024-03-02T20:00:00+05:30',
  'timeZone': 'Asia/Kolkata'},
 'end': {'dateTime': '2024-03-02T22:00:00+05:30', 'timeZone': 'Asia/Kolkata'},
 'iCalUID': '5mr1ike0hoq247uq55vrag4c9s@google.com',
 'sequence': 2,
 'reminders': {'useDefault': False,
  'overrides': [{'method': 'email', 'minutes': 1440},
   {'method': 'popup', 'minutes': 180}]},
 'eventType': 'default'}

In [14]:
mu.update_event_db(collection, updated_event[0]['id'], updated_event[0])

Updated event in DB


In [14]:
mu.get_event_db(collection, updated_event[0]['id'])

{'_id': ObjectId('65c53fbc22cc11c0ce8c5b26'),
 'kind': 'calendar#event',
 'etag': '"3414851391240000"',
 'id': '5mr1ike0hoq247uq55vrag4c9s',
 'status': 'confirmed',
 'htmlLink': 'https://www.google.com/calendar/event?eid=NW1yMWlrZTBob3EyNDd1cTU1dnJhZzRjOXMgZ3VudWJhbnNhbDEyOUBt',
 'created': '2024-02-08T20:54:55.000Z',
 'updated': '2024-02-08T20:54:55.620Z',
 'summary': 'Team meeting',
 'description': "A chance to hear more about Google's developer products.",
 'location': 'N/A',
 'creator': {'email': 'gunubansal129@gmail.com', 'self': True},
 'organizer': {'email': 'gunubansal129@gmail.com', 'self': True},
 'start': {'dateTime': '2024-03-02T20:00:00+05:30',
  'timeZone': 'Asia/Kolkata'},
 'end': {'dateTime': '2024-03-02T22:00:00+05:30', 'timeZone': 'Asia/Kolkata'},
 'iCalUID': '5mr1ike0hoq247uq55vrag4c9s@google.com',
 'sequence': 0,
 'reminders': {'useDefault': False,
  'overrides': [{'method': 'email', 'minutes': 1440},
   {'method': 'popup', 'minutes': 180}]},
 'eventType': 'default'

-----

### Delete event ##TDLR